In [ ]:
!pip install datasets sentence-transformers faiss-cpu transformers accelerate ragas

In [2]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [3]:
dataset['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator']}

# Data Preprocessing

In [4]:
import pandas as pd

df = pd.DataFrame(dataset['train'])

# Drop missing quotes
df = df.dropna(subset=['quote'])

# Normalize text
df['quote'] = df['quote'].str.lower()
df['author'] = df['author'].fillna("unknown").str.lower()
df['tags'] = df['tags'].apply(lambda x: x if isinstance(x, list) else [])

df.head()

,quote,author,tags
0,“be yourself; everyone else is already taken.”,oscar wilde,"[be-yourself, gilbert-perreira, honesty, inspi..."
1,"“i'm selfish, impatient and a little insecure....",marilyn monroe,"[best, life, love, mistakes, out-of-control, t..."
2,“two things are infinite: the universe and hum...,albert einstein,"[human-nature, humor, infinity, philosophy, sc..."
3,"“so many books, so little time.”",frank zappa,"[books, humor]"
4,“a room without books is like a body without a...,marcus tullius cicero,"[books, simile, soul]"


In [5]:
df['text_for_embedding'] = (
    "quote: " + df['quote'] +
    " | author: " + df['author'] +
    " | tags: " + df['tags'].apply(lambda x: ", ".join(x))
)

# Model Fine-Tuning

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

train_examples = []

for _, row in df.iterrows():
    train_examples.append(
        InputExample(
            texts=[row['quote'], " ".join(row['tags']) + " " + row['author']]
        )
    )

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [8]:
from sentence_transformers import losses

train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [9]:
model.save("fine_tuned_quote_model")

# Build the RAG Pipeline

In [10]:
import faiss
import numpy as np

embeddings = model.encode(
    df['text_for_embedding'].tolist(),
    show_progress_bar=True
)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

Batches:   0%|          | 0/79 [00:01<?, ?it/s]

In [11]:
def retrieve_quotes(query, k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)

    results = df.iloc[indices[0]][['quote', 'author', 'tags']]
    return results.to_dict(orient="records")

In [12]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="gpt2",
    max_new_tokens=200
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [13]:
def rag_answer(query):
    retrieved = retrieve_quotes(query)

    context = "\n".join(
        [f"{r['quote']} — {r['author']}" for r in retrieved]
    )

    prompt = f"""
Answer the question using the quotes below.

Context:
{context}

Question:
{query}

Answer:
"""

    output = generator(prompt)[0]["generated_text"]
    return output, retrieved

# RAG Evaluation

In [14]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

eval_data = {
    "question": [
        "quotes about hope by oscar wilde",
        "motivational quotes about success"
    ],
    "answer": [],
    "contexts": []
}

for q in eval_data["question"]:
    ans, ctx = rag_answer(q)
    eval_data["answer"].append(ans)
    eval_data["contexts"].append(
        [" ".join([c['quote'] for c in ctx])]
    )

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-2041930517.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/tmp/ipython-input-2041930517.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (

In [15]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.0 MB/s eta 0:00:00


In [16]:
!pip install streamlit sentence-transformers faiss-cpu transformers pyngrok

In [17]:
import faiss
import pickle

faiss.write_index(index, "quotes.faiss")

with open("quotes_metadata.pkl", "wb") as f:
    pickle.dump(df, f)

In [32]:
%%writefile app.py
import streamlit as st
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline


# Load Resources

@st.cache_resource
def load_all():
    model = SentenceTransformer("fine_tuned_quote_model")
    index = faiss.read_index("quotes.faiss")

    with open("quotes_metadata.pkl", "rb") as f:
        df = pickle.load(f)

    summarizer = pipeline(
        "summarization",
        model="facebook/bart-large-cnn"
    )

    return model, index, df, summarizer

model, index, df, summarizer = load_all()


# Retrieval Function

def retrieve_quotes(query, top_k):
    query_vec = model.encode([query])
    distances, indices = index.search(query_vec, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        row = df.iloc[idx]
        results.append({
            "quote": row["quote"],
            "author": row["author"],
            "tags": row["tags"],
            "similarity_score": float(distances[0][i])
        })
    return results


# Streamlit UI

st.set_page_config(page_title="Semantic Quote Retrieval", layout="wide")
st.title(" RAG-Based Semantic Quote Retrieval")

query = st.text_input(
    "Enter your query",
    placeholder="Show me quotes about courage by women authors"
)

top_k = st.slider("Number of results", 1, 10, 5)

show_scores = st.checkbox("Show similarity scores", value=True)
show_sources = st.checkbox("Show source quotes", value=True)


# Run Retrieval

if query:
    with st.spinner("Retrieving quotes..."):
        results = retrieve_quotes(query, top_k)

        combined_text = " ".join([r["quote"] for r in results])
        summary = summarizer(
            combined_text,
            max_length=80,
            min_length=30,
            do_sample=False
        )[0]["summary_text"]

    structured_response = {
        "query": query,
        "summary": summary,
        "results": [
            {
                "quote": r["quote"],
                "author": r["author"],
                "tags": r["tags"],
                "similarity_score": r["similarity_score"]
            }
            for r in results
        ]
    }

    st.subheader(" Structured JSON Response")
    st.json(structured_response)

    if show_sources:
        st.subheader(" Retrieved Quotes")
        for r in results:
            st.markdown(f"> *{r['quote']}*")
            st.markdown(f"**Author:** {r['author']}")
            st.markdown(f"**Tags:** {', '.join(r['tags'])}")
            if show_scores:
                st.markdown(f"**Similarity Score:** `{r['similarity_score']:.4f}`")
            st.divider()

Overwriting app.py


In [33]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

--2026-01-15 13:05:46--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2026-01-15 13:05:46--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-15T13%3A59%3A17Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-15

In [ ]:
!streamlit run app.py &>/content/logs.txt &
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501

2026-01-15T13:08:54Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-15T13:08:54Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-15T13:08:58Z INF +--------------------------------------------------------------------------------------------+
2026-01-15T13:08:58Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-15T13:08:58Z INF |  https://explaining-allow-travelling-presentations.try